In [7]:
!pip install lightgbm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 1.2 MB 7.4 MB/s eta 0:00:01


In [11]:
#coding:utf-8

import csv
import os
import shutil
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

g_ad_file_name = "https://lgb-1302229700.cos.ap-guangzhou.myqcloud.com/ad.csv?q-sign-algorithm=sha1&q-ak=AKIDXiH7zWGXSo6a0Rt_0rDY8KjuqHcbGF1jxF7QZDQ3lg7sFpk2kJB4eBX_3WXIF5go&q-sign-time=1592779160;1592782760&q-key-time=1592779160;1592782760&q-header-list=&q-url-param-list=&q-signature=aeff965bbe1bcde5d3cca4ad901281004966eed8&x-cos-security-token=cu63J8UAK0yBZoFpvfofFMNclpyz0Eotf377b8d995b48355fa55d2ee7b4a9e62VhuEO5tnMe5BObNkFNLRql9__8Tg8LFp96A8s6lRi_C_jAGPPkwpG-FdHsvddizHu2_kK5uvaWD1QcP0vgonCxqKL8ygNYwTvT3qONqFMxPa6epKhSSCL-npZMsv8snxx0YQR8gHAtLEQ_QbRMhm5C-EbVvASRPT7uVG9ub78iQ"



ad_data = pd.read_csv(g_ad_file_name)
ad_data = ad_data.loc[ad_data["product_id"] != "\\N"]
ad_data = ad_data.loc[ad_data["industry"] != "\\N"]

ad_data.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
9,20,20,34647,5,312,267
14,27,27,59,1,62948,254
21,46,46,1263,1,1274,247
33,63,63,66,1,62928,242
34,65,64,4573,3,22,141


In [12]:
g_click_file_name = "https://lgb-1302229700.cos.ap-guangzhou.myqcloud.com/click_log.csv?q-sign-algorithm=sha1&q-ak=AKIDgMFLq8bA6014l-YzzPXO5bKIja90tUFpeUjNhOBFUXugNgo6y8g9O33LDEp2GFuH&q-sign-time=1592779186;1592782786&q-key-time=1592779186;1592782786&q-header-list=&q-url-param-list=&q-signature=ccf5e832a43441f2bb510924051ca6dfd9e37947&x-cos-security-token=ljCIRq1AQncgI4Fk0dWn4fUobwJv4JQJ6ca1f7d2e00eacc00d0ed7a1dd521ab1zr4GVxsXIK0rXODJ66qpkbmtDlbpYqqKRFBGZVrRA0eCzLnVQrEIcHSd0EXGt77RfUA_k__NuzHW90Jc82VGrNmT-odqlE1bYUpZHctdlIhiutrUtPbw6sZabblKKqBDn_-eJkAe3xq_yMkFF8z_etoLn75_LHh0OEJdrVH2lrc"

click_data = pd.read_csv(g_click_file_name)
click_data = click_data.loc[click_data["click_times"] == 1]
click_data.head()   
len(click_data)

28426543

In [13]:
g_user_file_name = "https://lgb-1302229700.cos.ap-guangzhou.myqcloud.com/user.csv?q-sign-algorithm=sha1&q-ak=AKIDJ-K908GgnSwauAEYDu6XXMwbL_WaU1zQs3MTzCc-Nk_pHAqCBzW5aIlbichtkvtr&q-sign-time=1592779204;1592782804&q-key-time=1592779204;1592782804&q-header-list=&q-url-param-list=&q-signature=e0e81564ba3f2456e988dda4771b7b9f978cdfb8&x-cos-security-token=cu63J8UAK0yBZoFpvfofFMNclpyz0Eot378cbf77d9fa778ac667beb424ddcf84VhuEO5tnMe5BObNkFNLRqnIC75F5qOkfhK4KirbSIoKpnQ-DAovu_GjVyGQWx-7Jj5xrRBIhF4MME4qJkoDz_yFUl60AAE5WWR8vEKpqEGV6EoGN1hx2Ud1VllT6B0dKaaukUyekGLdDVbumPYeS--X7lE8CZbECtg4wsSDgyYs"    
user_data = pd.read_csv(g_user_file_name)
user_data.head()

,user_id,age,gender
0,1,4,1
1,2,10,1
2,3,7,2
3,4,5,1
4,5,4,1


In [14]:
a = pd.merge(ad_data, click_data, on="creative_id")

a.head()
len(a)

15376393

In [15]:
b = pd.merge(a, user_data, on="user_id")
b.head()
len(b)

15376393

In [49]:
blabel = b[['age','gender']].values
blabel.tolist()
print(blabel)


[[4 2]
 [4 2]
 [4 2]
 ...
 [5 2]
 [5 2]
 [4 2]]


In [27]:
btarget = b[['creative_id','ad_id','product_id','product_category','advertiser_id','industry','time','user_id','click_times']].values
btarget.tolist()
print(btarget)

[[20 20 '34647' ... 78 827182 1]
 [967 1509 '124' ... 66 827182 1]
 [226621 203400 '23' ... 2 827182 1]
 ...
 [4443818 3810508 '9140' ... 91 425815 1]
 [4444878 3811453 '34304' ... 91 755982 1]
 [4444878 3811453 '34304' ... 91 839485 1]]


In [50]:

from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold
import numpy as np

from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(btarget, blabel, test_size=0.3)
X_train=X_train.ravel()
X_test=X_test.ravel()
y_train=y_train.ravel()
y_test=y_test.ravel()
# 转换为Dataset数据格式
train_data = lgb.Dataset(X_train, label=y_train,feature_name=['c1', 'c2', 'c3','c4', 'c5', 'c6','c7', 'c8', 'c9','c10', 'c11'],categorical_feature=['c1', 'c2', 'c3','c4', 'c5', 'c6','c8'])
test_data = lgb.Dataset(X_test, label=y_test,feature_name=['c1', 'c2', 'c3','c4', 'c5', 'c6','c7', 'c8', 'c9','c10', 'c11'],categorical_feature=['c1', 'c2', 'c3','c4', 'c5', 'c6','c8'])


In [44]:
# 参数
params = {
    'learning_rate': 0.1,
    'lambda_l1': 0.1,
    'lambda_l2': 0.2,
    'max_depth': 5,
    'objective': 'multiclass',  # 目标函数
    'num_class': 3,
}



# 模型训练
print('a')
gbm = lgb.train(params, train_data, valid_sets=[test_data])



a


TypeError: Wrong type(ndarray) for label.
It should be list, numpy 1-D array or pandas Series

1

In [ ]:
# 模型预测
print('b')
y_pred = gbm.predict(X_test)
print(y_pred)
print('11111111111111111')
y_pred = [list(x).index(max(x)) for x in y_pred]
print(y_pred)

# 模型评估
print(accuracy_score(y_test, y_pred))

In [ ]:
from ti import session
ti_session = session.Session()
inputs = ti_session.upload_data(path="result_file", bucket="demo-project-ap-guangzhou-1259675134", key_prefix="contest")